In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from scipy import sparse as ssp
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")
import time

C:\Users\Himanshu\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
Path = "C:\\Users\\Himanshu\\Downloads\\train_aox2Jxw\\"
train_file = "train.csv"
test_file ="test_bqCt9Pv.csv"

train_df = pd.read_csv(Path+train_file)
test_df = pd.read_csv("C:\\Users\\Himanshu\\Downloads\\"+test_file)

# test_df.head()

# display(train_df.head())
# display(train_df.shape)
# display(train_df.isnull().sum())


In [3]:
train_df.loan_default.value_counts()

0    182543
1     50611
Name: loan_default, dtype: int64

In [3]:
##Creating New Variables


def total_months(x):
    l =[]
    l.append(x.split(' '))
    return int(l[0][0][0])*12 + int(l[0][1][0])

# 1. Age at loan disbursal date

train_df['Date.of.Birth'] = pd.to_datetime(train_df['Date.of.Birth'].values)
train_df['DisbursalDate'] = pd.to_datetime(train_df['DisbursalDate'].values)
train_df['Age_at_disbursaldate'] = (train_df['DisbursalDate'].dt.date - train_df['Date.of.Birth'].dt.date)
train_df['Age_at_disbursaldate'] = train_df['Age_at_disbursaldate'] / np.timedelta64(365, 'D')


train_df['Converted_AVERAGE.ACCT.AGE'] = train_df['AVERAGE.ACCT.AGE'].apply(total_months)
train_df['Converted_CREDIT.HISTORY.LENGTH'] = train_df['CREDIT.HISTORY.LENGTH'].apply(total_months)



df_supplier_id_ftg = pd.DataFrame(train_df.groupby('supplier_id')['loan_default'].apply(lambda x: x.sum()/x.count() if x.count() >=20 else 0)).rename(columns = {'loan_default': 'supplier_id_ftg'})
df_branch_id_ftg = pd.DataFrame(train_df.groupby('branch_id')['loan_default'].apply(lambda x: x.sum()/x.count() if x.count() >=20 else 0)).rename(columns = {'loan_default': 'branch_id_ftg'})
df_manufacturer_id_ftg = pd.DataFrame(train_df.groupby('manufacturer_id')['loan_default'].apply(lambda x: x.sum()/x.count() if x.count() >=20 else 0)).rename(columns = {'loan_default': 'manufacturer_id_ftg'})
df_pincode_id_ftg = pd.DataFrame(train_df.groupby('Current_pincode_ID')['loan_default'].apply(lambda x: x.sum()/x.count() if x.count() >=20  else 0)).rename(columns = {'loan_default': 'pincode_id_ftg'})
df_state_id_ftg = pd.DataFrame(train_df.groupby('State_ID')['loan_default'].apply(lambda x: x.sum()/x.count() if x.count() >=20  else 0)).rename(columns = {'loan_default': 'state_id_ftg'})
df_employee_code_id_ftg = pd.DataFrame(train_df.groupby('Employee_code_ID')['loan_default'].apply(lambda x: x.sum()/x.count() if x.count() >=20 else 0)).rename(columns = {'loan_default': 'employee_code_id_ftg'})

train_df['supplier_id_ftg'] = df_supplier_id_ftg.loc[train_df.supplier_id ,'supplier_id_ftg'].values
train_df['branch_id_ftg'] = df_branch_id_ftg.loc[train_df.branch_id ,'branch_id_ftg'].values
train_df['manufacturer_id_ftg'] = df_manufacturer_id_ftg.loc[train_df.manufacturer_id ,'manufacturer_id_ftg'].values
train_df['pincode_id_ftg'] = df_pincode_id_ftg.loc[train_df.Current_pincode_ID ,'pincode_id_ftg'].values
train_df['state_id_ftg'] = df_state_id_ftg.loc[train_df.State_ID ,'state_id_ftg'].values
train_df['employee_code_id_ftg'] = df_employee_code_id_ftg.loc[train_df.Employee_code_ID ,'employee_code_id_ftg'].values

train_df['Employment_ind'] = train_df['Employment.Type'].apply(lambda x: (1 if x == 'Self employed' else (0 if x == 'Salaried' else -1)))

#not using mobile avl flag since it is 1 for every one
train_df['Total_verification_flags'] = (train_df['Aadhar_flag'] + train_df['PAN_flag']+ train_df['VoterID_flag'] \
                                        +train_df['Driving_flag'] + train_df['Passport_flag'])


train_df.loc[((train_df['VoterID_flag'] == 0) & (train_df['PAN_flag'] == 0)),'VoterId_Pan_flag_Combo'] = 1
train_df.loc[((train_df['VoterID_flag'] == 0) & (train_df['PAN_flag'] == 1)),'VoterId_Pan_flag_Combo'] = 2
train_df.loc[((train_df['VoterID_flag'] == 1) & (train_df['PAN_flag'] == 0)),'VoterId_Pan_flag_Combo'] = 3
train_df.loc[((train_df['VoterID_flag'] == 1) & (train_df['PAN_flag'] == 1)),'VoterId_Pan_flag_Combo'] = 4

train_df['Disbursal_Month'] = train_df['DisbursalDate'].dt.month
train_df['Disbursal_Day'] = train_df['DisbursalDate'].dt.day


test_df['Date.of.Birth'] = pd.to_datetime(test_df['Date.of.Birth'].values)
test_df['DisbursalDate'] = pd.to_datetime(test_df['DisbursalDate'].values)
test_df['Age_at_disbursaldate'] = (test_df['DisbursalDate'].dt.date - test_df['Date.of.Birth'].dt.date)
test_df['Age_at_disbursaldate'] = test_df['Age_at_disbursaldate'] / np.timedelta64(365, 'D')
test_df['Age_at_disbursaldate'].head()

test_df['Converted_AVERAGE.ACCT.AGE'] = test_df['AVERAGE.ACCT.AGE'].apply(total_months)
test_df['Converted_CREDIT.HISTORY.LENGTH'] = test_df['CREDIT.HISTORY.LENGTH'].apply(total_months)


test_df['supplier_id_ftg'] = df_supplier_id_ftg.loc[test_df.supplier_id ,'supplier_id_ftg'].values
test_df['branch_id_ftg'] = df_branch_id_ftg.loc[test_df.branch_id ,'branch_id_ftg'].values
test_df['manufacturer_id_ftg'] = df_manufacturer_id_ftg.loc[test_df.manufacturer_id ,'manufacturer_id_ftg'].values
test_df['pincode_id_ftg'] = df_pincode_id_ftg.loc[test_df.Current_pincode_ID ,'pincode_id_ftg'].values
test_df['state_id_ftg'] = df_state_id_ftg.loc[test_df.State_ID ,'state_id_ftg'].values
test_df['employee_code_id_ftg'] = df_employee_code_id_ftg.loc[test_df.Employee_code_ID ,'employee_code_id_ftg'].values



test_df['Total_verification_flags'] = (test_df['Aadhar_flag'] + test_df['PAN_flag']+ test_df['VoterID_flag'] \
                                        +test_df['Driving_flag'] + test_df['Passport_flag'])

test_df['Employment_ind'] = test_df['Employment.Type'].apply(lambda x: (1 if x == 'Self employed' else (0 if x == 'Salaried' else -1)))

test_df.loc[((test_df['VoterID_flag'] == 0) & (test_df['PAN_flag'] == 0)),'VoterId_Pan_flag_Combo'] = 1
test_df.loc[((test_df['VoterID_flag'] == 0) & (test_df['PAN_flag'] == 1)),'VoterId_Pan_flag_Combo'] = 2
test_df.loc[((test_df['VoterID_flag'] == 1) & (test_df['PAN_flag'] == 0)),'VoterId_Pan_flag_Combo'] = 3
test_df.loc[((test_df['VoterID_flag'] == 1) & (test_df['PAN_flag'] == 1)),'VoterId_Pan_flag_Combo'] = 4

test_df['Disbursal_Month'] = test_df['DisbursalDate'].dt.month
test_df['Disbursal_Day'] = test_df['DisbursalDate'].dt.day    
# import matplotlib.pyplot as plt
# import seaborn as sns
# pd.crosstab(train_df['Employment.Type'], train_df.loan_default)

In [4]:
def tranformation(data):
    
    data['PRIMARY.INSTAL.AMT'] = np.log1p(data['PRIMARY.INSTAL.AMT'])
    data['SEC.INSTAL.AMT'] = np.log1p(data['SEC.INSTAL.AMT'])
    data['SEC.NO.OF.ACCTS'] = np.log1p(data['SEC.NO.OF.ACCTS'])
    data['SEC.ACTIVE.ACCTS'] = np.log1p(data['SEC.ACTIVE.ACCTS'])
    data['SEC.OVERDUE.ACCTS'] = np.log1p(data['SEC.OVERDUE.ACCTS'])
    #data['SEC.CURRENT.BALANCE'] = np.log1p(data['SEC.CURRENT.BALANCE'])
    data['SEC.SANCTIONED.AMOUNT'] = np.log1p(data['SEC.SANCTIONED.AMOUNT'])
    data['SEC.DISBURSED.AMOUNT'] = np.log1p(data['SEC.DISBURSED.AMOUNT'])

    #  filling  missing values in sec.current.balance
    data['SEC.CURRENT.BALANCE'].fillna(data['SEC.CURRENT.BALANCE'].mean(), inplace = True)

    data['PRI.NO.OF.ACCTS'] = np.log1p(data['PRI.NO.OF.ACCTS'])
    data['PRI.ACTIVE.ACCTS'] = np.log1p(data['PRI.ACTIVE.ACCTS'])
    data['PRI.OVERDUE.ACCTS'] = np.log1p(data['PRI.OVERDUE.ACCTS'])
    #data['PRI.CURRENT.BALANCE'] = np.log1p(data['PRI.CURRENT.BALANCE'])
    #data['PRI.SANCTIONED.AMOUNT'] = np.log1p(data['PRI.SANCTIONED.AMOUNT'])
    data['PRI.DISBURSED.AMOUNT'] = np.log1p(data['PRI.DISBURSED.AMOUNT'])


    #  filling  missing values in sec.current.balance
    data['PRI.CURRENT.BALANCE'].fillna(data['PRI.CURRENT.BALANCE'].mean(), inplace = True)
    data['PRI.SANCTIONED.AMOUNT'].fillna(data['PRI.SANCTIONED.AMOUNT'].mean(), inplace = True)
    data['PERFORM_CNS.SCORE'] = np.log1p(data['PERFORM_CNS.SCORE'])

    data['disbursed_amount'] = np.log1p(data['disbursed_amount'])
    data['ltv'] = np.log1p(data['ltv'])
    data['asset_cost'] = np.log1p(data['asset_cost'])


    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('No Bureau History Available', 0)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('Not Scored: Sufficient History Not Available', 0)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('Not Scored: Not Enough Info available on the customer', 0)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('Not Scored: No Activity seen on the customer (Inactive)',0)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('Not Scored: No Updates available in last 36 months', 0)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('Not Scored: Only a Guarantor', 0)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('Not Scored: More than 50 active Accounts found',0)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('M-Very High Risk', 1)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('L-Very High Risk', 1)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('K-High Risk', 2)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('J-High Risk', 2)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('I-Medium Risk', 3)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('H-Medium Risk', 3)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('G-Low Risk', 4)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('F-Low Risk', 4)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('E-Low Risk', 4)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('D-Very Low Risk', 5)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('C-Very Low Risk', 5)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('B-Very Low Risk', 5)
    data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace('A-Very Low Risk', 5)
    return data

In [5]:
train_df = tranformation(train_df)
test_df = tranformation(test_df)

### Model - 1  (with all the variables)

In [7]:
param = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'num_leaves': 255,  
    'max_depth': 12,  
    'min_child_samples':50,  
    'max_bin': 100,  
    'subsample': 0.7,  
    'subsample_freq': 1,  
    'colsample_bytree': 0.7,  
    'min_child_weight': 0,  
    'subsample_for_bin': 200,  
    'min_split_gain': 0,  
    'reg_alpha': 0,  
    'reg_lambda': 0,  
   # 'nthread': 8,
    'verbose': 5,
    'scale_pos_weight':99 
    }
        

                      

In [8]:
# Doing K fold crossvalidation

NFOLDS = 5
folds = StratifiedKFold(n_splits= NFOLDS, shuffle=True, random_state=218)

features = ['disbursed_amount', 'asset_cost', 'ltv', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',\
                'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS',
                'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
                'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE','employee_code_id_ftg','supplier_id_ftg',
               'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
               'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
               'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'NO.OF_INQUIRIES',
               'Age_at_disbursaldate',  'branch_id_ftg','pincode_id_ftg',
               'manufacturer_id_ftg',  'state_id_ftg', 'Converted_AVERAGE.ACCT.AGE',
               'Converted_CREDIT.HISTORY.LENGTH', 'Employment_ind','Total_verification_flags','VoterId_Pan_flag_Combo',
           'Disbursal_Month','Disbursal_Day','PERFORM_CNS.SCORE.DESCRIPTION']
#  

categorical_columns = ['VoterId_Pan_flag_Combo','Disbursal_Month','Employment_ind']
predictions1 = np.zeros(len(test_df))
feature_importance_df = pd.DataFrame()
start_time= time.time()
score = [0 for _ in range(folds.n_splits)]
oof = np.zeros(len(train_df))
max_iter = 5
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, train_df['loan_default'].values)):
    print("fold n : {}".format(fold_))
    
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][features],
                           label=train_df.iloc[trn_idx]['loan_default'], categorical_feature = categorical_columns
                          )
    val_data = lgb.Dataset(train_df.iloc[val_idx][features],
                           label=train_df.iloc[val_idx]['loan_default'] , categorical_feature = categorical_columns
                          )

    num_round = 10000
    clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=100,
                    early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    # we perform predictions by chunks
    current_pred = clf.predict(test_df[features].values)
    predictions1 += current_pred / min(folds.n_splits, max_iter)
   
    print("time elapsed: {:<5.2}min".format((time.time() - start_time) / 60))
    score[fold_] = roc_auc_score(train_df.iloc[val_idx]['loan_default'].values, oof[val_idx])
    if fold_ == max_iter - 1: break
        
if (folds.n_splits == max_iter):
    print("CV score: {:<8.5f}".format(metrics.roc_auc_score(train_df['loan_default'].values, oof)))
else:
     print("CV score: {:<8.5f}".format(sum(score) / max_iter))


fold n : 0
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.762723	valid_1's auc: 0.69187
[200]	training's auc: 0.812452	valid_1's auc: 0.691666
Early stopping, best iteration is:
[129]	training's auc: 0.779775	valid_1's auc: 0.692912
time elapsed: 0.43 min
fold n : 1
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.763211	valid_1's auc: 0.689123
[200]	training's auc: 0.812426	valid_1's auc: 0.688239
Early stopping, best iteration is:
[132]	training's auc: 0.782466	valid_1's auc: 0.690618
time elapsed: 0.78 min
fold n : 2
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.762987	valid_1's auc: 0.69269
[200]	training's auc: 0.813827	valid_1's auc: 0.694167
Early stopping, best iteration is:
[140]	training's auc: 0.786015	valid_1's auc: 0.694877
time elapsed: 1.2  min
fold n : 3
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.76119	valid_1

In [9]:
# feature_importance_df[feature_importance_df.fold == 5].sort_values(by = 'importance', ascending = False)

###  Model - 2 (excluding the ftg variables)

In [11]:
# Doing K fold crossvalidation

NFOLDS = 5
folds = StratifiedKFold(n_splits= NFOLDS, shuffle=True, random_state=218)

features2 = ['disbursed_amount', 'asset_cost', 'ltv', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',\
                'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS',
                'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
                'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
               'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
               'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
               'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'NO.OF_INQUIRIES',
               'Age_at_disbursaldate',  'Converted_AVERAGE.ACCT.AGE',
               'Converted_CREDIT.HISTORY.LENGTH', 'Employment_ind','Total_verification_flags','VoterId_Pan_flag_Combo',
           'Disbursal_Month','Disbursal_Day','PERFORM_CNS.SCORE.DESCRIPTION']
#  

categorical_columns = ['VoterId_Pan_flag_Combo','Disbursal_Month','Employment_ind']
predictions2 = np.zeros(len(test_df))
feature_importance_df = pd.DataFrame()
start_time= time.time()
score = [0 for _ in range(folds.n_splits)]
oof = np.zeros(len(train_df))
max_iter = 5
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, train_df['loan_default'].values)):
    print("fold n : {}".format(fold_))
    
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][features2],
                           label=train_df.iloc[trn_idx]['loan_default'], categorical_feature = categorical_columns
                          )
    val_data = lgb.Dataset(train_df.iloc[val_idx][features2],
                           label=train_df.iloc[val_idx]['loan_default'] , categorical_feature = categorical_columns
                          )

    num_round = 10000
    clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=100,
                    early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features2], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features2
    fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    # we perform predictions by chunks
    current_pred = clf.predict(test_df[features2].values)
    predictions2 += current_pred / min(folds.n_splits, max_iter)
   
    print("time elapsed: {:<5.2}min".format((time.time() - start_time) / 60))
    score[fold_] = roc_auc_score(train_df.iloc[val_idx]['loan_default'].values, oof[val_idx])
    if fold_ == max_iter - 1: break
        
if (folds.n_splits == max_iter):
    print("CV score: {:<8.5f}".format(metrics.roc_auc_score(train_df['loan_default'].values, oof)))
else:
     print("CV score: {:<8.5f}".format(sum(score) / max_iter))


fold n : 0
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.708791	valid_1's auc: 0.640288
[200]	training's auc: 0.761265	valid_1's auc: 0.638341
Early stopping, best iteration is:
[122]	training's auc: 0.722742	valid_1's auc: 0.640724
time elapsed: 0.5  min
fold n : 1
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.708213	valid_1's auc: 0.637754
[200]	training's auc: 0.760448	valid_1's auc: 0.633701
Early stopping, best iteration is:
[115]	training's auc: 0.718476	valid_1's auc: 0.638326
time elapsed: 0.98 min
fold n : 2
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.709188	valid_1's auc: 0.640178
[200]	training's auc: 0.760758	valid_1's auc: 0.63569
Early stopping, best iteration is:
[106]	training's auc: 0.712893	valid_1's auc: 0.640624
time elapsed: 1.4  min
fold n : 3
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.708363	valid

In [ ]:
### Model -3 Only ftg variables

In [13]:
# Doing K fold crossvalidation

NFOLDS = 5
folds = StratifiedKFold(n_splits= NFOLDS, shuffle=True, random_state=218)

features3 = ['employee_code_id_ftg','supplier_id_ftg','branch_id_ftg','pincode_id_ftg',
               'manufacturer_id_ftg',  'state_id_ftg']
#  

categorical_columns = ['VoterId_Pan_flag_Combo','Disbursal_Month','Employment_ind']
predictions3 = np.zeros(len(test_df))
feature_importance_df = pd.DataFrame()
start_time= time.time()
score = [0 for _ in range(folds.n_splits)]
oof = np.zeros(len(train_df))
max_iter = 5
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, train_df['loan_default'].values)):
    print("fold n : {}".format(fold_))
    
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][features2],
                           label=train_df.iloc[trn_idx]['loan_default']
                          )
    val_data = lgb.Dataset(train_df.iloc[val_idx][features2],
                           label=train_df.iloc[val_idx]['loan_default'] 
                          )

    num_round = 10000
    clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=100,
                    early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features2], num_iteration=clf.best_iteration)
    
#     fold_importance_df = pd.DataFrame()
#     fold_importance_df["feature"] = features3
#     fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
#     fold_importance_df["fold"] = fold_ + 1
#     feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    # we perform predictions by chunks
    current_pred = clf.predict(test_df[features3].values)
    predictions3 += current_pred / min(folds.n_splits, max_iter)
   
    print("time elapsed: {:<5.2}min".format((time.time() - start_time) / 60))
    score[fold_] = roc_auc_score(train_df.iloc[val_idx]['loan_default'].values, oof[val_idx])
    if fold_ == max_iter - 1: break
        
if (folds.n_splits == max_iter):
    print("CV score: {:<8.5f}".format(metrics.roc_auc_score(train_df['loan_default'].values, oof)))
else:
     print("CV score: {:<8.5f}".format(sum(score) / max_iter))


fold n : 0
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.708783	valid_1's auc: 0.641571
[200]	training's auc: 0.759974	valid_1's auc: 0.638828
Early stopping, best iteration is:
[125]	training's auc: 0.725054	valid_1's auc: 0.642197
time elapsed: 0.52 min
fold n : 1
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.707804	valid_1's auc: 0.6375
[200]	training's auc: 0.759281	valid_1's auc: 0.635216
Early stopping, best iteration is:
[107]	training's auc: 0.712142	valid_1's auc: 0.638051
time elapsed: 0.91 min
fold n : 2
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.708789	valid_1's auc: 0.639016
[200]	training's auc: 0.760687	valid_1's auc: 0.635934
Early stopping, best iteration is:
[112]	training's auc: 0.716746	valid_1's auc: 0.639638
time elapsed: 1.3  min
fold n : 3
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.707045	valid_

In [16]:

# print(roc_auc_score(Y_test, final_scores))
submission = pd.read_csv("C:\\Users\\Himanshu\\Downloads\\"+test_file)
submission['loan_default'] = (0.6*predictions1+0.25*predictions2+0.15*predictions3)
submission[['UniqueID','loan_default']].to_csv('D2_6th_solution.csv', index = False)

#### TRying out different technique

In [50]:
train_df['VoterId_Pan_flag_Combo'] = train_df['VoterId_Pan_flag_Combo'].astype('category')
train_df['Disbursal_Month'] = train_df['Disbursal_Month'].astype('category')
train_df['Employment_ind'] = train_df['Employment_ind'].astype('category')

test_df['VoterId_Pan_flag_Combo'] = test_df['VoterId_Pan_flag_Combo'].astype('category')
test_df['Disbursal_Month'] = test_df['Disbursal_Month'].astype('category')
test_df['Employment_ind'] = test_df['Employment_ind'].astype('category')

# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()
# data['branch_id'] = le.fit_transform(data['branch_id'])
# data['manufacturer_id'] = le.fit_transform(data['manufacturer_id'])
# data['State_ID'] = le.fit_transform(data['State_ID'])

#  ['VoterId_Pan_flag_Combo','Disbursal_Month','Employment_ind']

### Getting dummies for state, supplier id, zip etc 

In [6]:
test_df['loan_default'] = 0
data = pd.concat([train_df, test_df])
print(train_df.shape)
data.drop(columns = ['loan_default'], inplace = True)
print(data.shape)

(233154, 55)
(345546, 54)


In [28]:
data.head()

,AVERAGE.ACCT.AGE,Aadhar_flag,Age_at_disbursaldate,CREDIT.HISTORY.LENGTH,Converted_AVERAGE.ACCT.AGE,Converted_CREDIT.HISTORY.LENGTH,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,Date.of.Birth,DisbursalDate,Disbursal_Day,...,manuf_id__49,manuf_id__51,manuf_id__67,manuf_id__86,manuf_id__120,manuf_id__145,manuf_id__152,manuf_id__153,manuf_id__155,manuf_id__156
0,0yrs 0mon,1,34.205479,0yrs 0mon,0,0,0,1984-01-01,2018-03-08,8,...,0,0,0,0,0,0,0,0,0,0
1,1yrs 11mon,1,33.178082,1yrs 11mon,13,13,1,1985-07-31,2018-09-26,26,...,0,0,0,0,0,0,0,0,0,0
2,0yrs 0mon,1,32.397260,0yrs 0mon,0,0,0,1985-08-24,2018-01-08,8,...,0,0,0,0,0,0,0,0,0,0
3,0yrs 8mon,1,24.838356,1yrs 3mon,8,15,0,1993-12-30,2018-10-26,26,...,0,0,0,0,0,0,0,0,0,0
4,0yrs 0mon,1,41.065753,0yrs 0mon,0,0,0,1977-09-12,2018-09-26,26,...,0,0,0,0,0,0,0,0,0,0


In [14]:
data  = pd.get_dummies(data, prefix = 'pin_id_', columns= ['Current_pincode_ID'])
data  = pd.get_dummies(data, prefix = 'supp_id_', columns= ['supplier_id'])
data  = pd.get_dummies(data, prefix = 'manuf_id_', columns= ['manufacturer_id'])
# data  = pd.get_dummies(data, prefix = 'st_id_', columns= ['State_ID'])
# data  = pd.get_dummies(data, prefix = 'brnch_id_', columns= ['branch_id'])
# data  = pd.get_dummies(data, prefix = 'emp_id_', columns= ['Employee_code_ID'])



In [15]:
data.shape

(345546, 10248)

In [ ]:
data = pd.concat([data,pin_code,supp_id , manf_id, st_id, brnch_id, emp_id])
data.shape

In [24]:

final_features = ['disbursed_amount', 'asset_cost', 'ltv', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',\
                'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS',
                'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
                'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
               'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
               'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
               'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'NO.OF_INQUIRIES',
               'Age_at_disbursaldate',  'Converted_AVERAGE.ACCT.AGE',
               'Converted_CREDIT.HISTORY.LENGTH', 'Employment_ind','Total_verification_flags','VoterId_Pan_flag_Combo',
           'Disbursal_Month','Disbursal_Day','PERFORM_CNS.SCORE.DESCRIPTION']+ list(data.columns[51:])

In [33]:
train_data = data.iloc[:233154,:]
test_data = data.iloc[233154:,:]
train_data['loan_default'] = train_df['loan_default']

In [38]:
del [data,train_df, test_df]
import gc
gc.collect()

In [35]:
test_data.shape

(112392, 10248)

In [41]:
# Doing K fold crossvalidation

NFOLDS = 5
folds = StratifiedKFold(n_splits= NFOLDS, shuffle=True, random_state=218)

features2 = final_features

categorical_columns = ['VoterId_Pan_flag_Combo','Disbursal_Month','Employment_ind']
predictions4 = np.zeros(len(test_data))
feature_importance_df = pd.DataFrame()
start_time= time.time()
score = [0 for _ in range(folds.n_splits)]
oof = np.zeros(len(train_data))
max_iter = 5
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_data.values, train_data['loan_default'].values)):
    print("fold n : {}".format(fold_))
    
    trn_data = lgb.Dataset(train_data.iloc[trn_idx][features2],
                           label=train_data.iloc[trn_idx]['loan_default'], categorical_feature = categorical_columns
                          )
    val_data = lgb.Dataset(train_data.iloc[val_idx][features2],
                           label=train_data.iloc[val_idx]['loan_default'] , categorical_feature = categorical_columns
                          )

    num_round = 10000
    clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=100,
                    early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(train_data.iloc[val_idx][features2], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features2
    fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    # we perform predictions by chunks
    current_pred = clf.predict(test_data[features2].values)
    predictions2 += current_pred / min(folds.n_splits, max_iter)
   
    print("time elapsed: {:<5.2}min".format((time.time() - start_time) / 60))
    score[fold_] = roc_auc_score(train_data.iloc[val_idx]['loan_default'].values, oof[val_idx])
    if fold_ == max_iter - 1: break
        
if (folds.n_splits == max_iter):
    print("CV score: {:<8.5f}".format(metrics.roc_auc_score(train_data['loan_default'].values, oof)))
else:
     print("CV score: {:<8.5f}".format(sum(score) / max_iter))


MemoryError: 

In [42]:
train_data.to_csv('train_dummy.csv')
test_data.to_csv('test_dummy.csv')

In [43]:
pwd

'C:\\Users\\Himanshu'